In [70]:
%reload_kedro

[05/06/23 12:15:12] INFO     Resolved project path as: C:\Work\Weather_Aus\weather-aus.             ]8;id=170209;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=379219;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/06/23 12:15:23] WARNING  Failed to send data to Heap. Exception of type 'ConnectionError' was     ]8;id=179711;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=373310;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro_telemetry\plugin.py#219\219]8;;\
                             raised.                                                                               

                    INFO     Kedro project Weather_aus                                              ]8;id=954159;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=752923;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=30933;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=815423;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=896416;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=511675;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\ipython\__init__.py#110\110]8;;\

In [71]:
catalog.list()

['weather_AUS_raw',
 'df1',
 'df1_treat_missing_value',
 'X_training',
 'y_training',
 'X_training0',
 'df1_scaled_data1',
 'model',
 'parameters']

In [72]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

import warnings

warnings.filterwarnings('ignore')

In [73]:
df = catalog.load('weather_AUS_raw')

                    INFO     Loading data from 'weather_AUS_raw' (CSVDataSet)...                ]8;id=846003;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=193983;file://c:\Work\Weather_Aus\venv\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [74]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [41]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
df.shape

(145460, 23)

# We need to create nodes

In [132]:
def extract_training_data(df):
    """ Extracting training data from raw data and droping Date column
    Arg: Pandas DataFrame here raw data
    Return: DataFrame i.e. Training data"""
    df = df.drop('Date',axis=1)
    df1 = df[df['RainTomorrow'].notna()]
    return df1

In [133]:
df1 = extract_training_data(df)

In [134]:
def treat_missing_val(df1):
    """ Misssing value treatment on training data
    Arg: df1
    Return: df1_treat_missing_value"""
    df1_treat_missing_value = df1.fillna(method='ffill',axis=0).fillna(method='bfill',axis=0)
    return df1_treat_missing_value

In [135]:
df1_treat_missing_value = treat_missing_val(df1)

In [136]:
def train_data_split(df1_treat_missing_value):
    """ Splitting training data in X,y
    Arg: df1
    Return: X_training,y_training"""
    X_training = df1_treat_missing_value.drop(['RainTomorrow'],axis=1)
    y_training = df1_treat_missing_value['RainTomorrow']
    return X_training
    return y_training

In [48]:
X_training = train_data_split(df1_treat_missing_value)
y_training = train_data_split(df1_treat_missing_value)

In [137]:
type(X_training)

pandas.core.frame.DataFrame

In [50]:
#X_training = pd.Series(X_training)

In [51]:
#X_training = pd.DataFrame(X_training)

In [52]:
#y_training = pd.Series(y_training)
#y_training = pd.DataFrame(y_training)

In [138]:
type(y_training)

pandas.core.frame.DataFrame

In [54]:
# X_training = df1_treat_missing_value.drop(['RainTomorrow'],axis=1)
# y_training = df1_treat_missing_value['RainTomorrow'

In [55]:
# def label_encoding(df1_treat_missing_value):
#     """ Label encoding converting categorical variables to numerical variables
#     Arg: df1_treat_missing_value
#     Return: df1_label_encoder"""
#     from sklearn.preprocessing import LabelEncoder
#     l_encoder = LabelEncoder()
#     df1_treat_missing_value['Location'] = l_encoder.fit_transform(df1_treat_missing_value['Location'])
#     df1_treat_missing_value['WindGustDir'] = l_encoder.fit_transform(df1_treat_missing_value['WindGustDir'])
#     df1_treat_missing_value['WindDir9am'] = l_encoder.fit_transform(df1_treat_missing_value['WindDir9am'])
#     df1_treat_missing_value['WindDir3pm'] = l_encoder.fit_transform(df1_treat_missing_value['WindDir3pm'])
#     df1_treat_missing_value['RainToday'] = l_encoder.fit_transform(df1_treat_missing_value['RainToday'])
#     return df1_treat_missing_value

In [139]:
def label_encoding(X_training):
    """ Label encoding converting categorical variables to numerical variables
    Arg: df1_treat_missing_value
    Return: df1_label_encoder"""
    from sklearn.preprocessing import LabelEncoder
    l_encoder = LabelEncoder()
    X_training['Location'] = l_encoder.fit_transform(X_training['Location'])
    X_training['WindGustDir'] = l_encoder.fit_transform(X_training['WindGustDir'])
    X_training['WindDir9am'] = l_encoder.fit_transform(X_training['WindDir9am'])
    X_training['WindDir3pm'] = l_encoder.fit_transform(X_training['WindDir3pm'])
    X_training['RainToday'] = l_encoder.fit_transform(X_training['RainToday'])
    X_training0 = X_training.copy()
    return X_training0

In [140]:
X_training0 = label_encoding(X_training)

In [141]:
X_training0

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,2,13.4,22.9,0.6,12.0,12.3,13,44.0,13,14,...,24.0,71.0,22.0,1007.7,1007.1,8.0,2.0,16.9,21.8,0
1,2,7.4,25.1,0.0,12.0,12.3,14,44.0,6,15,...,22.0,44.0,25.0,1010.6,1007.8,8.0,2.0,17.2,24.3,0
2,2,12.9,25.7,0.0,12.0,12.3,15,46.0,13,15,...,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,0
3,2,9.2,28.0,0.0,12.0,12.3,4,24.0,9,0,...,9.0,45.0,16.0,1017.6,1012.8,8.0,2.0,18.1,26.5,0
4,2,17.5,32.3,1.0,12.0,12.3,13,41.0,1,7,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,41,3.5,21.8,0.0,6.4,10.7,0,31.0,2,0,...,13.0,59.0,27.0,1024.7,1021.2,8.0,7.0,9.4,20.9,0
145455,41,2.8,23.4,0.0,6.4,10.7,0,31.0,9,1,...,11.0,51.0,24.0,1024.6,1020.3,8.0,7.0,10.1,22.4,0
145456,41,3.6,25.3,0.0,6.4,10.7,6,22.0,9,3,...,9.0,56.0,21.0,1023.5,1019.1,8.0,7.0,10.9,24.5,0
145457,41,5.4,26.9,0.0,6.4,10.7,3,37.0,9,14,...,9.0,53.0,24.0,1021.0,1016.8,8.0,7.0,12.5,26.1,0


In [147]:
def scaling_data(X_training0):
    """ Scaling data is used to standardized data.
    Arg: df1_label_encoder
    Return: df1_scaled_data"""
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df1_scaled_data = scaler.fit_transform(X_training0)
    df1_scaled_data1 = pd.DataFrame(df1_scaled_data)
    return df1_scaled_data
    return df1_scaled_data1

In [148]:
df1_scaled_data = scaling_data(X_training0)

In [149]:
type(df1_scaled_data)

numpy.ndarray

In [150]:
df1_scaled_data1 = scaling_data(X_training0)

In [151]:
type(df1_scaled_data1)

numpy.ndarray

In [152]:
def train_test_splitting(df1_scaled_data1,y_training):
    """ data split into training and testing
    Args: df1_scaled_data,y_training
    Return: X_train,X_test,y_train,y_test"""

    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test = train_test_split(df1_scaled_data1,y_training,random_state=0,test_size=0.20)
    return X_train,X_test,y_train,y_test
# X_train = train_test_splitting(df1_scaled_data1,y_training)
# X_test = train_test_splitting(df1_scaled_data1,y_training)
# y_train = train_test_splitting(df1_scaled_data1,y_training)
# y_test = train_test_splitting(df1_scaled_data1,y_training)

In [157]:
X_train = train_test_splitting(df1_scaled_data1,y_training)
X_test = train_test_splitting(df1_scaled_data1,y_training)
y_train = train_test_splitting(df1_scaled_data1,y_training)
y_test = train_test_splitting(df1_scaled_data1,y_training)

In [153]:
X_train

0    [[1.5634358297378823, 0.26717084653716766, -0....
1    [[-1.1055808168950956, 0.8137014192853136, 0.8...
2                Location  MinTemp  MaxTemp  Rainfa...
3                 Location  MinTemp  MaxTemp  Rainf...
dtype: object

In [154]:
y_train

0    [[1.5634358297378823, 0.26717084653716766, -0....
1    [[-1.1055808168950956, 0.8137014192853136, 0.8...
2                Location  MinTemp  MaxTemp  Rainfa...
3                 Location  MinTemp  MaxTemp  Rainf...
dtype: object

In [155]:
X_test

(array([[ 1.56343583,  0.26717085, -0.08816116, ...,  0.28200287,
         -0.18631331, -0.53717035],
        [-1.10558082,  1.59445938,  0.86716057, ...,  1.44922523,
          0.83373439,  1.86160685],
        [-1.45676722, -0.38866584,  1.82248231, ...,  1.7717472 ,
          1.91124956, -0.53717035],
        ...,
        [-1.66747906, -0.04513234, -0.79060362, ..., -0.37839926,
         -0.66041999,  1.86160685],
        [ 0.50987663, -0.70096903,  0.3894997 , ..., -0.39375745,
          0.41709518, -0.53717035],
        [ 1.63367311,  1.21969556,  1.1621864 , ...,  1.40315066,
          0.15849154, -0.53717035]]),
 array([[-1.10558082,  0.81370142,  0.81096518, ...,  1.40315066,
          0.87683499, -0.53717035],
        [-1.24605538, -0.84150546, -1.24016679, ..., -1.00808501,
         -1.29256223, -0.53717035],
        [ 1.35272399, -0.3262052 , -1.0996783 , ..., -0.71627942,
         -1.24946162,  1.86160685],
        ...,
        [-0.4734453 , -0.77904482,  0.05232733, ..., -

In [156]:
y_test

(array([[ 1.56343583,  0.26717085, -0.08816116, ...,  0.28200287,
         -0.18631331, -0.53717035],
        [-1.10558082,  1.59445938,  0.86716057, ...,  1.44922523,
          0.83373439,  1.86160685],
        [-1.45676722, -0.38866584,  1.82248231, ...,  1.7717472 ,
          1.91124956, -0.53717035],
        ...,
        [-1.66747906, -0.04513234, -0.79060362, ..., -0.37839926,
         -0.66041999,  1.86160685],
        [ 0.50987663, -0.70096903,  0.3894997 , ..., -0.39375745,
          0.41709518, -0.53717035],
        [ 1.63367311,  1.21969556,  1.1621864 , ...,  1.40315066,
          0.15849154, -0.53717035]]),
 array([[-1.10558082,  0.81370142,  0.81096518, ...,  1.40315066,
          0.87683499, -0.53717035],
        [-1.24605538, -0.84150546, -1.24016679, ..., -1.00808501,
         -1.29256223, -0.53717035],
        [ 1.35272399, -0.3262052 , -1.0996783 , ..., -0.71627942,
         -1.24946162,  1.86160685],
        ...,
        [-0.4734453 , -0.77904482,  0.05232733, ..., -

In [130]:
y_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 y_test.shape                                                                                 │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'shape'

In [64]:
X_test.shape

(28439, 21)

In [106]:
X_test = np.array(X_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_test = np.array(y_test)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 X_test = np.array(X_test)                                                                    │
│   2 X_train = np.array(X_train)                                                                  │
│   3 y_train = np.array(y_train)                                                                  │
│   4 y_test = np.array(y_test)                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 
dimensions. The detected shape was (4,) + inhomogeneous part.

In [160]:
def logregAlgorithm(X_train,X_test,y_train,y_test):
    """Logistic regression algorithm used
    Args: X_train,X_test,y_train,y_test
    Return:y_pred_train,y_pred_test"""
    from sklearn.linear_model import LogisticRegression
    logreg = LogisticRegression(solver='liblinear',random_state=0)
    model = logreg.fit(X_train,y_train)
    # import pickle
    # with open('Logistic_Regression.pkl','w') as f:
    #  pickle.dump(model,f)
    # return model
model = logregAlgorithm(X_train,X_test,y_train,y_test)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   # with open('Logistic_Regression.pkl','w') as f:                                        │
│   10 │   #  pickle.dump(model,f)                                                                 │
│   11 │   # return model                                                                          │
│ ❱ 12 model = logregAlgorithm(X_train,X_test,y_train,y_test)                                      │
│   13                                                                                             │
│                                                                                                  │
│ in logregAlgorithm:7                                                                             │
│                                                                                                  │
│    4 │   Return:y_pred_train,y_pred_test"""                                                      │
│    5 │   from sklearn.linear_model import LogisticRegression                                     │
│    6 │   logreg = LogisticRegression(solver='liblinear',random_state=0)                          │
│ ❱  7 │   model = logreg.fit(X_train,y_train)                                                     │
│    8 │   # import pickle                                                                         │
│    9 │   # with open('Logistic_Regression.pkl','w') as f:                                        │
│   10 │   #  pickle.dump(model,f)                                                                 │
│                                                                                                  │
│ c:\Work\Weather_Aus\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1196 in fit         │
│                                                                                                  │
│   1193 │   │   else:                                                                             │
│   1194 │   │   │   _dtype = [np.float64, np.float32]                                             │
│   1195 │   │                                                                                     │
│ ❱ 1196 │   │   X, y = self._validate_data(                                                       │
│   1197 │   │   │   X,                                                                            │
│   1198 │   │   │   y,                                                                            │
│   1199 │   │   │   accept_sparse="csr",                                                          │
│                                                                                                  │
│ c:\Work\Weather_Aus\venv\lib\site-packages\sklearn\base.py:584 in _validate_data                 │
│                                                                                                  │
│    581 │   │   │   │   │   check_y_params = {**default_check_params, **check_y_params}           │
│    582 │   │   │   │   y = check_array(y, input_name="y", **check_y_params)                      │
│    583 │   │   │   else:                                                                         │
│ ❱  584 │   │   │   │   X, y = check_X_y(X, y, **check_params)                                    │
│    585 │   │   │   out = X, y                                                                    │
│    586 │   │                                                                                     │
│    587 │   │   if not no_val_X and check_params.get("ensure_2d", True):                          │
│                                                                                                  │
│ c:\Work\Weather_Aus\venv\lib\site-packages\sklearn\utils\va

In [98]:
model = logregAlgorithm(X_train,X_test,y_train,y_test)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model = logregAlgorithm(X_train,X_test,y_train,y_test)                                       │
│   2                                                                                              │
│                                                                                                  │
│ in logregAlgorithm:7                                                                             │
│                                                                                                  │
│    4 │   Return:y_pred_train,y_pred_test"""                                                      │
│    5 │   from sklearn.linear_model import LogisticRegression                                     │
│    6 │   logreg = LogisticRegression(solver='liblinear',random_state=0)                          │
│ ❱  7 │   model = logreg.fit(X_train,y_train)                                                     │
│    8 │   import pickle                                                                           │
│    9 │   with open('Logistic_Regression.pkl','w') as f:                                          │
│   10 │    pickle.dump(model,f)                                                                   │
│                                                                                                  │
│ c:\Work\Weather_Aus\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1196 in fit         │
│                                                                                                  │
│   1193 │   │   else:                                                                             │
│   1194 │   │   │   _dtype = [np.float64, np.float32]                                             │
│   1195 │   │                                                                                     │
│ ❱ 1196 │   │   X, y = self._validate_data(                                                       │
│   1197 │   │   │   X,                                                                            │
│   1198 │   │   │   y,                                                                            │
│   1199 │   │   │   accept_sparse="csr",                                                          │
│                                                                                                  │
│ c:\Work\Weather_Aus\venv\lib\site-packages\sklearn\base.py:584 in _validate_data                 │
│                                                                                                  │
│    581 │   │   │   │   │   check_y_params = {**default_check_params, **check_y_params}           │
│    582 │   │   │   │   y = check_array(y, input_name="y", **check_y_params)                      │
│    583 │   │   │   else:                                                                         │
│ ❱  584 │   │   │   │   X, y = check_X_y(X, y, **check_params)                                    │
│    585 │   │   │   out = X, y                                                                    │
│    586 │   │                                                                                     │
│    587 │   │   if not no_val_X and check_params.get("ensure_2d", True):                          │
│                                                                                                  │
│ c:\Work\Weather_Aus\venv\lib\site-packages\sklearn\utils\validation.py:1122 in check_X_y         │
│                                                                                                  │
│   1119 │   │   input_name="X",                                                                   │
│   1120 │   )                                               

In [90]:
def prediction(X_train,X_test,y_train,y_test,logreg):
    """Prediction is used
    Args: X_train,X_test,y_train,y_test,model
    Return: y_pred_train,y_pred_test"""
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    return y_pred_train
    return y_pred_test

In [94]:
y_pred_train= prediction(X_train,X_test,y_train,y_test,model)
y_pred_test = prediction(X_train,X_test,y_train,y_test,model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 y_pred_train= prediction(X_train,X_test,y_train,y_test,model)                                │
│   2 y_pred_test = prediction(X_train,X_test,y_train,y_test,model)                                │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [65]:
def evaluation(y_pred_train,y_pred_test):
    """Model accuracy check
    Args: y_pred_train,y_pred_test
    Return: Accuracy_score"""
    from sklearn.metrics import accuracy_score
    accuracy_score_train = accuracy_score(y_train,y_pred_train)
    accuracy_score_test = accuracy_score(y_test,y_pred_test)
    return accuracy_score_train,accuracy_score_test
accuracy_score_train,accuracy_score_test = evaluation(y_pred_train,y_pred_test)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   accuracy_score_train = accuracy_score(y_train,y_pred_train)                             │
│    7 │   accuracy_score_test = accuracy_score(y_test,y_pred_test)                                │
│    8 │   return accuracy_score_train,accuracy_score_test                                         │
│ ❱  9 accuracy_score_train,accuracy_score_test = evaluation(y_pred_train,y_pred_test)             │
│   10                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y_pred_train' is not defined

In [78]:
def extracting_inference_data(df):
    """ Inference data extraction
    Arg: df
    Return: df2 """
    df2 = df[df["RainTomorrow"].isna()]
    return df2
df2 = extracting_inference_data(df)

In [79]:
def splitting_inference_data(df2):
    """ Splitting inference data
    Arg: df2
    Return: X_inf,y_inf """
    X_inf = df2.drop(['RainTomorrow'],axis=1)
    y_inf = df2['RainTomorrow']
    return X_inf,y_inf
X_inf,y_inf = splitting_inference_data(df2)

In [ ]:
def inference_data_treat_missing_val(X_inf):
    """ Misssing value treatment on training data
    Arg: df1
    Return: df1_treat_missing_value"""
    Xinf_treat_missing_value = X_inf.fillna(method='ffill',axis=0).fillna(method='bfill',axis=0)
    return Xinf_treat_missing_value
Xinf_treat_missing_value = inference_data_treat_missing_val(X_inf)

In [ ]:
def inference_data_label_encoding(Xinf_treat_missing_value):
    """ Label encoding converting categorical variables to numerical variables
    Arg: df1_treat_missing_value
    Return: df1_label_encoder"""
    from sklearn.preprocessing import LabelEncoder
    l_encoder = LabelEncoder()
    Xinf_treat_missing_value['Location'] = l_encoder.fit_transform(Xinf_treat_missing_value['Location'])
    Xinf_treat_missing_value['WindGustDir'] = l_encoder.fit_transform(Xinf_treat_missing_value['WindGustDir'])
    Xinf_treat_missing_value['WindDir9am'] = l_encoder.fit_transform(Xinf_treat_missing_value['WindDir9am'])
    Xinf_treat_missing_value['WindDir3pm'] = l_encoder.fit_transformXinf_treat_missing_value['WindDir3pm'])
    Xinf_treat_missing_value['RainToday'] = l_encoder.fit_transform(Xinf_treat_missing_value['RainToday'])
    Xinf_treat_missing_value0 = Xinf_treat_missing_value.copy()
    return Xinf_treat_missing_value0

In [ ]:
Xinf_treat_missing_value0 = inference_data_label_encoding(Xinf_treat_missing_value)

In [ ]:
def inference_data_scaling(Xinf_treat_missing_value0):
    """ Scaling data is used to standardized data.
    Arg: df1_label_encoder
    Return: df1_scaled_data"""
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    inference_scaled_data = scaler.fit_transform(Xinf_treat_missing_value0)
    df1_inference_scaled_data = pd.DataFrame(inference_scaled_data)
    return df1_inference_scaled_data

In [ ]:
df1_inference_scaled_data = inference_data_scaling(Xinf_treat_missing_value0)

In [ ]:
def logregAlgorithm(X_train,X_test,y_train,y_test):
    """Logistic regression algorithm used
    Args: X_train,X_test,y_train,y_test
    Return:y_pred_train,y_pred_test"""
    from sklearn.linear_model import LogisticRegression
    logreg = LogisticRegression(solver='liblinear',random_state=0)
    logreg.fit(X_train,y_train)
    y_pred_inf = logreg.predict(df1_inference_scaled_data)
    return y_pred_inf